In [ ]:
%

In [1]:
import sagemaker
from sagemaker.pytorch import PyTorchModel

# 1. Sesión y rol
session = sagemaker.Session()
region = session.boto_region_name  # debería ser "us-east-1"
role = "arn:aws:iam::057176724057:role/Rol_ML_Cuantico_FullAccess"  # tu rol de SageMaker

# 2. Ruta al TAR del modelo en S3 (la que YA tienes)
s3_model_path = "s3://modelos-sagemaker/modelos/mnist/mnist_quantum/model_hybrid.tar.gz"

# 3. Definir el modelo en script mode
pytorch_model = PyTorchModel(
    model_data=s3_model_path,
    role=role,
    framework_version="1.13",           # coherente con el contenedor que usabas
    py_version="py39",
    entry_point="inference_quantum.py", # archivo dentro de source_dir
    source_dir="modelos/code/mnist/mnist_quantum/code",  # ruta LOCAL donde tienes inference_quantum.py y modelcnn.py
)

# 4. Desplegar el endpoint
predictor = pytorch_model.deploy(
    endpoint_name="mnist-hybrid-v2",    # nombre nuevo para no chocar con el viejo
    initial_instance_count=1,
    instance_type="ml.m5.large",        # o el que estabas usando
)

print("Endpoint creado:", predictor.endpoint_name)


TypeError: Metaclasses with custom tp_new are not supported.

In [2]:
import boto3
import botocore
import json
import base64
from botocore.config import Config

AWS_REGION = "us-east-1"
ENDPOINT_NAME = "mnist-hybrid"
TEST_IMAGE_PATH = "test_digit.png"  # pon aquí tu imagen de prueba

# Config con timeout para que no se quede “pegado” infinito
boto_config = Config(
    read_timeout=120,      # tiempo máximo esperando la respuesta
    retries={"max_attempts": 3}
)

runtime = boto3.client("sagemaker-runtime", region_name=AWS_REGION, config=boto_config)

# 1. Leer la imagen y convertirla a base64
with open(TEST_IMAGE_PATH, "rb") as f:
    image_bytes = f.read()

image_b64 = base64.b64encode(image_bytes).decode("utf-8")

# 2. Payload EXACTO que espera tu input_fn: {"input": "<b64>"}
payload = {"input": image_b64}

print("Invocando endpoint:", ENDPOINT_NAME)

try:
    response = runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType="application/json",
        Accept="application/json",
        Body=json.dumps(payload),
    )

    status_code = response["ResponseMetadata"]["HTTPStatusCode"]
    body_str = response["Body"].read().decode("utf-8")

    print(f"\nHTTP status: {status_code}")
    print("Respuesta cruda:")
    print(body_str)

    try:
        body_json = json.loads(body_str)
        print("\nJSON parseado:")
        print(json.dumps(body_json, indent=2, ensure_ascii=False))
    except json.JSONDecodeError:
        print("\n(No es JSON válido, se muestra como texto)")

except botocore.exceptions.ClientError as e:
    print("\n❌ Error al invocar el endpoint:")
    print("Código:", e.response["Error"]["Code"])
    print("Mensaje:", e.response["Error"]["Message"])
    print("RequestId:", e.response["ResponseMetadata"].get("RequestId"))
    raise
except botocore.exceptions.EndpointConnectionError as e:
    print("\n❌ Problema de conexión con SageMaker Runtime:")
    print(e)
    raise


Invocando endpoint: mnist-hybrid

❌ Error al invocar el endpoint:
Código: ModelError
Mensaje: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/mnist-hybrid in account 057176724057 for more information.
RequestId: 33443322-673e-43bd-b22e-7145fe127898


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/mnist-hybrid in account 057176724057 for more information.

# Navega al directorio que contiene la carpeta de tu modelo
# cd /ruta/a/la/carpeta/padre

# Crea el archivo .tar.gz usando el flag -h para dereferenciar los enlaces simbólicos
tar -h -czvf model.tar.gz my_model_dir/


In [1]:
import boto3

AWS_REGION = "us-east-1"
ENDPOINT_NAME = "mnist-hybrid"

sm = boto3.client("sagemaker", region_name=AWS_REGION)

resp = sm.describe_endpoint(EndpointName=ENDPOINT_NAME)
print("Estado:", resp["EndpointStatus"])
print("Último mensaje:", resp.get("FailureReason", "OK"))


Estado: InService
Último mensaje: OK
